In [1]:
# 필요 라이브러리 설치
!pip install konlpy pandas seaborn gensim wordcloud python-mecab-ko wget

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 580.9/580.9 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 17.5 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=0cc42f241a7964dee84d3a7303e0b159dfd20d2ee9d7044792451a2ccbc62f18
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [2]:
# 라이브러리 로
from mecab import MeCab
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import wget,os
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import nltk
import wget,os

In [3]:
!sudo apt-get install -y fonts-nanum

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 18 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (10.6 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 124950 files and direc

In [4]:
os.environ['JAVA_HOME'] = "C:\Program Files\Java\jdk-19"

In [5]:
FONT_PATH = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_name = fm.FontProperties(fname=FONT_PATH, size=10).get_name()
print(font_name)
plt.rcParams['font.family']=font_name
assert plt.rcParams['font.family'] == [font_name], "한글 폰트가 설정되지 않았습니다."

NanumGothic


In [6]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- train 데이터 전처리

In [7]:
data = pd.read_csv('/content/drive/MyDrive/Type_of_Inquiry_Classification/train.csv')
data.shape

(3706, 2)

In [8]:
data

,text,label
0,"self.convs1 = nn.ModuleList([nn.Conv2d(1, Co, ...",코드2
1,현재 이미지를 여러개 업로드 하기 위해 자바스크립트로 동적으로 폼 여러개 생성하는데...,웹
2,glob.glob(PATH) 를 사용할 때 질문입니다.\n\nPATH에 [ ] 가 ...,코드2
3,"tmpp = tmp.groupby(by = 'Addr1', as_index=Fals...",코드2
4,filename = TEST_IMAGE + str(round(frame_sec)) ...,코드2
...,...,...
3701,"토큰화 이후 train val 를 분리하고 각 train set, val set에 ...",코드2
3702,올린 값들 중 최고점인 건가요? 아니면 최근에 올린 파일로 무조건 갱신인가요?\n최...,시스템 운영
3703,수업에서 cacoo랑 packet tracer를 배우는 이유가\n\n1. IT 인프...,이론
3704,inplace =True 해도 값이 변경이 안되고 none으로 뜹니다. 혹시 원격지...,원격


In [9]:
def mecab_tokenizer(text):
    pos = mecab.pos(text)
    return [word_pos[0] for word_pos in pos if word_pos[1] in ("NNG", 'NNP','SL')]

In [10]:
mecab = MeCab()

data['string'] = data['text'].apply(mecab_tokenizer)

In [11]:
label_dict = {
    '코드1': 0,
    '코드2': 0,
    '웹': 1,
    '이론': 2,
    '시스템 운영': 3,
    '원격': 4
}

In [12]:
data = data.replace({'label' : label_dict})

<ipython-input-12-02279b70eb50>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace({'label' : label_dict})


In [13]:
data

,text,label,string
0,"self.convs1 = nn.ModuleList([nn.Conv2d(1, Co, ...",0,"[self, convs, nn, ModuleList, nn, Conv, d, Co,..."
1,현재 이미지를 여러개 업로드 하기 위해 자바스크립트로 동적으로 폼 여러개 생성하는데...,1,"[이미지, 업로드, 자바, 스크립트, 동적, 폼, 생성, 클릭, 기본, 예제, 코드..."
2,glob.glob(PATH) 를 사용할 때 질문입니다.\n\nPATH에 [ ] 가 ...,0,"[glob, glob, PATH, 사용, 때, 질문, PATH, 포함, 작동, 질문..."
3,"tmpp = tmp.groupby(by = 'Addr1', as_index=Fals...",0,"[tmpp, tmp, groupby, by, Addr, as, index, Fals..."
4,filename = TEST_IMAGE + str(round(frame_sec)) ...,0,"[filename, TEST, IMAGE, str, round, frame, sec..."
...,...,...,...
3701,"토큰화 이후 train val 를 분리하고 각 train set, val set에 ...",0,"[토큰, 이후, train, val, 분리, train, set, val, set,..."
3702,올린 값들 중 최고점인 건가요? 아니면 최근에 올린 파일로 무조건 갱신인가요?\n최...,3,"[값, 최고점, 최근, 파일, 갱신, 최고점, 값, 경우, 변경]"
3703,수업에서 cacoo랑 packet tracer를 배우는 이유가\n\n1. IT 인프...,2,"[수업, cacoo, packet, tracer, 이유, IT, 인프라, 구조, 이..."
3704,inplace =True 해도 값이 변경이 안되고 none으로 뜹니다. 혹시 원격지...,4,"[inplace, True, 값, 변경, none, 원격, 지원, 가능]"


In [14]:
# 토큰화
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

stop_words = ['은', '는', '이', '가', '을', '를', '안녕', '감사', 'ㅠㅠ', '습니다', 'ㅎㅎ', '합니다',
                      '금일', '안녕', '일차', '간과', '가요', '감사', '구합', '차', 'eda', 'ㄱ', 'ㄴ', 'ㄷ',
                      'ㅅ', 'ㅁ', 'ㄷ', 'ㅂ', '후', '시', '거', '의',  '그', '때', '로', '요', '과', '걸', '중', '안',
                      '은', '제', '왜', '결', '수', '컴', '터', '제하', '고생']

tokenizer = Tokenizer(num_words=40000)
tokenizer.fit_on_texts(data['string'])
sequences = tokenizer.texts_to_sequences(data['string'])
# 불용어 처리
sequences = [[w for w in s if w not in stop_words] for s in sequences]

# 패딩
max_length = max([len(item) for item in sequences])
# padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='pre', truncating='post')
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')


# 벡터화
x = padded_sequences

# 라벨 인코딩
# encoder = LabelEncoder()
# encoder.fit(data['label'])
# y = encoder.transform(data['label'])
y = data['label']

In [15]:
max_length

836

In [16]:
from tensorflow.keras.utils import to_categorical

y = to_categorical(y)

In [17]:
y

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.]])

- test 데이터 전처리

In [22]:
test = pd.read_csv('/content/drive/MyDrive/Type_of_Inquiry_Classification/test.csv')

In [23]:
# 토큰화
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# tokenizer = Tokenizer(num_words=40000) train에는 해줘야하지만 test(강사님 test.csv)에는 해주는 거 아님!
# tokenizer.fit_on_texts(test['text'])
sequences = tokenizer.texts_to_sequences(test['text'])

# 불용어 처리
# sequences = [[w for w in s if w not in stop_words] for s in sequences]

# 패딩
max_length = 836
# padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='pre', truncating='post')
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')


# 벡터화
test = padded_sequences

In [24]:
test

array([[   0,    0,    0, ...,    0,    0,  447],
       [   0,    0,    0, ...,    0,    0,   60],
       [   0,    0,    0, ...,  982,   10,  982],
       ...,
       [   0,    0,    0, ...,  969, 5761,  108],
       [   0,    0,    0, ...,   39, 5948,  491],
       [   0,    0,    0, ...,  509, 1945, 6131]], dtype=int32)

In [25]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size  = 0.2, random_state = 2023)

In [26]:
max_words = 40000
embedding_dim = 128
max_len = 836

- DNN 모델링 1

In [53]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Flatten

In [28]:
# 세션 클리어
keras.backend.clear_session()

# 레이어 사슬처럼 엮기
il = keras.layers.Input( shape = (max_len, ) )

el = Embedding(max_words + 1,
                        embedding_dim,
                        input_length = max_len)(il)
# hl = keras.layers.Dropout(0.2)(el)

hl = Conv1D(64, 5, activation = 'relu', padding = 'same')(el)
hl = Conv1D(64, 5, activation = 'relu', padding = 'same')(hl)
pl = GlobalMaxPooling1D()(hl)
hl = keras.layers.Dropout(0.2)(pl)

hl = Dense(32, activation = 'relu')(hl)

ol = Dense(5, activation = 'softmax')(hl)

# 모델 처음과 끝 지정
model = Model(il, ol)

# 컴파일
model.compile(loss = 'categorical_crossentropy',
                        metrics = ['accuracy'],
                        optimizer = 'adam')
# 요약
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 836)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 836, 128)            │       5,120,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 836, 64)             │          41,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 836, 64)             │          20,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d                 │ (None, 64)                  │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │             165 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,183,941 (19.78 MB)

 Trainable params: 5,183,941 (19.78 MB)

 Non-trainable params: 0 (0.00 B)

In [32]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=7,
                   verbose=1)

In [33]:
model.fit(x_train, y_train, epochs=100, batch_size= 16,
          validation_split=0.2, verbose =1,
          callbacks=[es])

Epoch 1/100
149/149 ━━━━━━━━━━━━━━━━━━━━ 25s 171ms/step - accuracy: 0.9941 - loss: 0.0223 - val_accuracy: 0.7555 - val_loss: 1.4284
Epoch 2/100
149/149 ━━━━━━━━━━━━━━━━━━━━ 42s 178ms/step - accuracy: 0.9851 - loss: 0.0493 - val_accuracy: 0.7589 - val_loss: 1.4865
Epoch 3/100
149/149 ━━━━━━━━━━━━━━━━━━━━ 41s 179ms/step - accuracy: 0.9843 - loss: 0.0372 - val_accuracy: 0.7504 - val_loss: 1.4876
Epoch 4/100
149/149 ━━━━━━━━━━━━━━━━━━━━ 41s 179ms/step - accuracy: 0.9865 - loss: 0.0379 - val_accuracy: 0.7875 - val_loss: 1.4164
Epoch 5/100
149/149 ━━━━━━━━━━━━━━━━━━━━ 41s 180ms/step - accuracy: 0.9967 - loss: 0.0088 - val_accuracy: 0.7622 - val_loss: 1.5013
Epoch 6/100
149/149 ━━━━━━━━━━━━━━━━━━━━ 40s 174ms/step - accuracy: 0.9989 - loss: 0.0056 - val_accuracy: 0.7723 - val_loss: 1.5325
Epoch 7/100
149/149 ━━━━━━━━━━━━━━━━━━━━ 41s 175ms/step - accuracy: 0.9970 - loss: 0.0079 - val_accuracy: 0.7605 - val_loss: 1.6840
Epoch 8/100
149/149 ━━━━━━━━━━━━━━━━━━━━ 42s 181ms/step - accuracy: 0.9984 -

In [41]:
pred = model.predict(x_val)

# print(f1_score(y_test, pred.argmax(axis = 1), average = 'micro'))

24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step


In [42]:
pred_arg = np.argmax(pred, axis = 1)
len(pred_arg)

742

In [43]:
pred.argmax(axis = 1)

array([2, 1, 2, 0, 2, 0, 4, 0, 0, 3, 2, 2, 3, 1, 0, 3, 3, 2, 3, 2, 0, 1,
       0, 0, 0, 0, 0, 1, 3, 2, 0, 4, 0, 2, 1, 2, 2, 0, 2, 3, 2, 0, 2, 2,
       0, 3, 0, 2, 0, 3, 3, 3, 0, 0, 0, 2, 0, 3, 1, 0, 3, 1, 1, 1, 2, 0,
       3, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 3, 1, 2, 0, 0, 3, 2, 0, 0, 3,
       4, 2, 2, 3, 3, 0, 0, 3, 1, 1, 4, 2, 0, 2, 3, 3, 2, 2, 0, 0, 0, 2,
       0, 1, 2, 0, 1, 0, 0, 1, 1, 1, 3, 0, 2, 3, 0, 1, 3, 2, 3, 0, 2, 2,
       0, 2, 0, 3, 0, 0, 0, 2, 0, 0, 0, 0, 3, 2, 1, 0, 0, 3, 3, 0, 2, 0,
       2, 0, 0, 0, 0, 0, 4, 3, 0, 2, 0, 1, 3, 0, 3, 0, 0, 3, 2, 1, 0, 3,
       2, 0, 0, 1, 2, 3, 1, 1, 0, 4, 0, 2, 0, 3, 0, 0, 0, 3, 1, 3, 0, 2,
       0, 1, 2, 2, 3, 3, 2, 3, 2, 3, 0, 2, 0, 0, 1, 0, 2, 0, 3, 3, 1, 0,
       0, 1, 3, 1, 1, 0, 2, 0, 4, 3, 3, 3, 0, 3, 3, 3, 2, 0, 0, 1, 2, 0,
       0, 3, 0, 2, 0, 0, 3, 2, 0, 1, 0, 2, 0, 3, 3, 0, 1, 3, 2, 0, 3, 0,
       3, 0, 0, 3, 0, 1, 0, 2, 0, 2, 3, 2, 3, 1, 1, 1, 2, 3, 2, 3, 3, 0,
       3, 2, 0, 0, 3, 1, 0, 3, 2, 3, 0, 2, 1, 0, 1,

In [44]:
len(y_val)

742

In [45]:
from sklearn.metrics import f1_score

print(f1_score(y_val.argmax(axis = 1), pred.argmax(axis = 1), average = 'micro'))

0.7789757412398922


- DNN 모델링 2

In [54]:
# 세션 클리어
keras.backend.clear_session()

# 레이어 사슬처럼 엮기
il = keras.layers.Input( shape = (max_len, ) )

el = Embedding(max_words + 1,
                        embedding_dim,
                        input_length = max_len)(il)

hl = Conv1D(64, 5, activation = 'relu', padding = 'same')(el)
pl = GlobalMaxPooling1D()(hl)
dl = Dropout(0.2)(pl)
fl = Flatten()(dl)
hl = Dense(32, activation = 'relu')(fl)

ol = Dense(5, activation = 'softmax')(hl)

# 모델 처음과 끝 지정
model = Model(il, ol)

# 컴파일
model.compile(loss = 'categorical_crossentropy',
                        metrics = ['accuracy'],
                        optimizer = 'adam')
# 요약
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 836)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 836, 128)            │       5,120,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 836, 64)             │          41,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d                 │ (None, 64)                  │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │             165 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,163,397 (19.70 MB)

 Trainable params: 5,163,397 (19.70 MB)

 Non-trainable params: 0 (0.00 B)

In [55]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=7,
                   verbose=1)

In [56]:
model.fit(x_train, y_train, epochs=100, batch_size= 16,
          validation_split=0.2, verbose =1,
          callbacks=[es])

Epoch 1/100
149/149 ━━━━━━━━━━━━━━━━━━━━ 22s 135ms/step - accuracy: 0.3993 - loss: 1.4133 - val_accuracy: 0.6121 - val_loss: 0.9799
Epoch 2/100
149/149 ━━━━━━━━━━━━━━━━━━━━ 21s 137ms/step - accuracy: 0.7885 - loss: 0.6525 - val_accuracy: 0.7960 - val_loss: 0.5677
Epoch 3/100
149/149 ━━━━━━━━━━━━━━━━━━━━ 22s 143ms/step - accuracy: 0.9588 - loss: 0.1775 - val_accuracy: 0.8027 - val_loss: 0.5710
Epoch 4/100
149/149 ━━━━━━━━━━━━━━━━━━━━ 41s 147ms/step - accuracy: 0.9750 - loss: 0.0923 - val_accuracy: 0.7909 - val_loss: 0.6572
Epoch 5/100
149/149 ━━━━━━━━━━━━━━━━━━━━ 41s 148ms/step - accuracy: 0.9929 - loss: 0.0398 - val_accuracy: 0.7926 - val_loss: 0.6930
Epoch 6/100
149/149 ━━━━━━━━━━━━━━━━━━━━ 40s 141ms/step - accuracy: 0.9983 - loss: 0.0155 - val_accuracy: 0.7943 - val_loss: 0.7369
Epoch 7/100
149/149 ━━━━━━━━━━━━━━━━━━━━ 41s 138ms/step - accuracy: 0.9966 - loss: 0.0206 - val_accuracy: 0.8027 - val_loss: 0.7649
Epoch 8/100
149/149 ━━━━━━━━━━━━━━━━━━━━ 41s 138ms/step - accuracy: 0.9975 -

In [57]:
pred = model.predict(x_val)

# print(f1_score(y_test, pred.argmax(axis = 1), average = 'micro'))

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step


In [58]:
from sklearn.metrics import f1_score

print(f1_score(y_val.argmax(axis = 1), pred.argmax(axis = 1), average = 'micro'))

0.8328840970350404


- test 데이터 예측

In [59]:
pred = model.predict(test)

30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step


In [69]:
pred.argmax(axis = 1)

array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 1, 0, 0, 0, 0, 3, 0, 0,
       0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0,
       0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 0, 1, 0, 1, 3, 0, 0, 1, 0, 0, 3,
       0, 0, 0, 1, 3, 2, 0, 1, 0, 0, 1, 3, 0, 2, 0, 0, 2, 0, 0, 0, 0, 3,
       2, 3, 0, 3, 2, 0, 0, 0, 3, 0, 0, 0, 2, 3, 0, 1, 3, 1, 1, 0, 0, 0,
       0, 3, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 1, 0, 0, 3, 0, 1, 0, 3, 0, 3,
       1, 3, 0, 0, 0, 0, 0, 3, 0, 0, 1, 4, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 2, 0, 0, 2, 2, 0, 1, 0, 2, 2, 3, 2, 3, 2, 1, 3, 0, 0, 0, 1, 2,
       1, 3, 2, 3, 2, 2, 2, 0, 1, 2, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 3, 1, 3, 3, 3, 3, 3, 3, 0, 0, 1, 3, 2, 0, 0, 3,
       1, 1, 2, 1, 0, 0, 2, 0, 0, 0, 4, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 3, 2, 0, 1, 3, 1, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 1, 1, 0,
       0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 4, 3, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 3, 1, 0, 0, 0, 2, 3, 0, 3, 2, 0,

In [70]:
pred_arg = np.argmax(pred, axis = 1)
len(pred_arg)

929

In [74]:
test_sub = pd.read_csv('/content/drive/MyDrive/Type_of_Inquiry_Classification/test.csv')

In [75]:
test_sub = test_sub.drop('text', axis = 1)

In [76]:
test_sub['label'] = pred_arg

In [77]:
test_sub

,id,label
0,0,1
1,1,1
2,2,0
3,3,0
4,4,0
...,...,...
924,924,1
925,925,2
926,926,0
927,927,1


In [78]:
test_sub.to_csv('/content/drive/MyDrive/Type_of_Inquiry_Classification/test_sub.csv', index = False)